Illumina 双端测序 + index reads（I1/I2） 的典型结构：

..._I1_001.fastq.gz：Index read 1（i7）

..._I2_001.fastq.gz：Index read 2（i5，只有用到双索引时才需要）

..._L1_1.fq.gz：Read 1（R1）

..._L1_2.fq.gz：Read 2（R2）

做分析一般用 R1/R2；I1/I2主要用于拆分样本（demux）或上游软件识别 barcode/index。

0) 确认cellrange可用
        cellranger --version
   设置 reference 路径
   reference 目录里应该有什么（用来验证下对不对）

    对 GEX reference，通常能看到类似这些（版本略有差异，但会有关键文件/目录）：
    fasta/
    genes/
    star/（或 star/Genome 等）
    reference.json
    以及一些 idx/metadata 文件
1) 建目录并把每个样本的 R1/R2 软链接进去
    mkdir -p /mnt/e/Gao/gex_fastq/{MR1-Mid-0-3GEX,MR1-Mid-80-3GEX,MV4-11-ITD5-3GEX,MV4-TP53fs-2-5GEX} #创建分级目录

    给原始 FASTQ 文件起一个“新名字”放到新目录里，cellranger 最稳定支持的命名模式是：<SAMPLE>_S1_L001_R1_001.fastq.gz <SAMPLE>_S1_L001_R2_001.fastq.gz
    但不复制文件、不占额外磁盘空间，只是创建一个软链接（快捷方式），
    让 cellranger 能按标准 10x 命名规则识别这个文件是 R1。
    ln -sf /mnt/e/Gao/MR1-Mid-0-3GEX_L1_1.fq.gz /mnt/e/Gao/gex_fastq/MR1-Mid-0-3GEX/MR1-Mid-0-3GEX_S1_L001_R1_001.fastq.gz
    ln -sf /mnt/e/Gao/MR1-Mid-0-3GEX_L1_2.fq.gz /mnt/e/Gao/gex_fastq/MR1-Mid-0-3GEX/MR1-Mid-0-3GEX_S1_L001_R2_001.fastq.gz
    ln -sf /mnt/e/Gao/MR1-Mid-80-3GEX_L1_1.fq.gz /mnt/e/Gao/gex_fastq/MR1-Mid-80-3GEX/MR1-Mid-80-3GEX_S1_L001_R1_001.fastq.gz
    ln -sf /mnt/e/Gao/MR1-Mid-80-3GEX_L1_2.fq.gz /mnt/e/Gao/gex_fastq/MR1-Mid-80-3GEX/MR1-Mid-80-3GEX_S1_L001_R2_001.fastq.gz
    ln -sf /mnt/e/Gao/MV4-11-ITD5-3GEX_L1_1.fq.gz /mnt/e/Gao/gex_fastq/MV4-11-ITD5-3GEX/MV4-11-ITD5-3GEX_S1_L001_R1_001.fastq.gz
    ln -sf /mnt/e/Gao/MV4-11-ITD5-3GEX_L1_2.fq.gz /mnt/e/Gao/gex_fastq/MV4-11-ITD5-3GEX/MV4-11-ITD5-3GEX_S1_L001_R2_001.fastq.gz
    ln -sf /mnt/e/Gao/MV4-TP53fs-2-5GEX_L1_1.fq.gz /mnt/e/Gao/gex_fastq/MV4-TP53fs-2-5GEX/MV4-TP53fs-2-5GEX_S1_L001_R1_001.fastq.gz
    ln -sf /mnt/e/Gao/MV4-TP53fs-2-5GEX_L1_2.fq.gz /mnt/e/Gao/gex_fastq/MV4-TP53fs-2-5GEX/MV4-TP53fs-2-5GEX_S1_L001_R2_001.fastq.gz

2) 先快速确认 cellranger 能“看到”FASTQ
    find /mnt/e/Gao/gex_fastq -maxdepth 2 -type l -o -type f | head -n 20

3) 跑 cellranger count（每个样本一条命令）

    cellranger count \
    --id=GEX_MR1_Mid_0_3GEX \ 
    --transcriptome=/path/to/refdata-gex-GRCh38-2020-A \
    --fastqs=/mnt/e/Gao/gex_fastq/MR1-Mid-0-3GEX \
    --sample=MR1-Mid-0-3GEX \
    --localcores=16 --localmem=64


    cellranger count \
    --id=GEX_MR1_Mid_0_3GEX \
    --transcriptome=/mnt/e/Gao/ref/refdata-gex-GRCh38-2024-A \
    --fastqs=/mnt/e/Gao/gex_fastq/MR1-Mid-0-3GEX \
    --sample=MR1-Mid-0-3GEX \
    --localcores=16 --localmem=64
4) 跑完后你要看什么（QC & 输出）
    每个 --id/outs/ 里最关键：
    web_summary.html（总体 QC：reads、mapping、median genes/UMIs、fraction in cells 等）
    filtered_feature_bc_matrix/（下游分析用的矩阵）
    raw_feature_bc_matrix/
    possorted_genome_bam.bam（需要时做更深层分析）

reference 

reference 在https://www.10xgenomics.com/support/software/cell-ranger/downloads下载

Human reference (GRCh38) —— 给 GEX（转录组）用的
    包含：
    全基因组 FASTA
    全基因注释 GTF
    为 基因表达定量 优化的索引
    适合做：
    scRNA-seq 表达矩阵
    UMI counting
    基因层面的表达分析
    但它：
    ❌ 不包含专门为 TCR/BCR V(D)J 重排 优化的参考结构
    ❌ 不能正确组装 TCR/BCR 序列
    ❌ 不能用于 cellranger vdj

Human reference (GRCh38/Ensembl/10x) for Cell Ranger V(D)J —— 给 TCR/BCR 用的
    包含：
    TCR / BCR 的 V / D / J / C 基因片段
    为 重排序列组装（de novo assembly） 设计的 reference 结构
    专用的 reference.json，描述免疫受体基因模型
    适合做：
    TCR α/β 链重构
    BCR 重链/轻链重构
    clonotype 定义
    CDR3 提取
    productive / non-productive 注释